In [4]:
"""Opportunistic Graph Search"""

import numpy as np
import random
import math

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [5]:
def random_node(r=10, center=(0,0)):
    alpha = 2 * math.pi * random.random()
    
    # random radius
    r = r * math.sqrt(random.random())
    
    # calculating coordinates
    x = r * math.cos(alpha) + center[0]
    y = r * math.sin(alpha) + center[1]
    
    return (x, y)

In [191]:
def init_paths(num_riders, hub_coord):
    riders = ['r{}'.format(x) for x in range(num_riders)]
    paths = {}
    for rider in riders:
        paths[rider] = {
            'nodes_coord': [hub_coord],
            'time_tracker': [],
            'path': [],
            'must_leave': float('inf'),
            'will_return': float('inf'),
            'active': False,
            'completed': []
        }
        
    return paths

In [192]:
def compute_time_matrix(nodes):
    tm = [[0 for x in range(len(nodes))] for y in range(len(nodes))]
    
    for row, start in enumerate(nodes):
        for col, end in enumerate(nodes):
            x = np.array(start)
            y = np.array(end)
            distance = round(np.linalg.norm(x - y)) # Round to nearest integer
            
            if (end == (0,0)) | (row == col):
                tm[row][col] = distance 
            else:
                tm[row][col] = distance + 5 # We account for the handling time in the time to execute the path
       
    return tm

In [193]:
def compute_time_windows(time_windows, time, inputs):
    # From the moment an order is received the rider must wait for the order to be prepared at the hub.
    # We always fix the preparation time as the minimum wait time. 
    updated_time_windows = [(5,5)] 
    for idx, window in enumerate(time_windows):
        if idx == 0:
            continue
        
        # Prevents optimizer from accounting for preparation time while on the road.
        updated_time_windows.append((inptus['prep_time'], window[1] - time_delta))
    
    updated_time_windows.append((inptus['prep_time'], inptus['delivery_max']))
    
    return updated_time_windows

In [194]:
def find_path(data):
    """Solve the VRP with time windows."""
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']), data["num_vehicles"], data["depot"])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        100,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    
    # Add penalties to drop nodes
    penalty = 100
    for node in range(1, len(data['time_matrix'])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)
    
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data["depot"]:
            continue
        index = manager.NodeToIndex(location_idx)
       ######## print('Time window',time_window)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # Add time window constraints for each vehicle start node.
    depot_idx = data["depot"]
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][depot_idx][0],
            data['time_windows'][depot_idx][1])

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    #if solution:
     #   print_solution(data, manager, routing, solution) 
    return manager, routing, solution

In [195]:
def print_order(time, coord):
    print('\n// NEW ORDER\n')
    print('Order time: {}'.format(time))
    print('Node coord: {}\n'.format(coord))

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    time_dimension = routing.GetDimensionOrDie('Time')
    total_nodes = -1 # Takes into account first node
    nodes_visited = []
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            total_nodes += 1
            if manager.IndexToNode(index) != 0:
                nodes_visited.append(manager.IndexToNode(index))
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            index = solution.Value(routing.NextVar(index))
        print("INDEX", index)
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
    print('Nodes visited: {}'.format(nodes_visited))
    print('Total number of nodes: {}'.format(total_nodes))

In [196]:
'Run on every new order'
def assign_path(paths, node_coord, time, inputs, verbose=False):
    if verbose: print('\n////// ASSIGN PATH')
    order_assigned = False
    for rider, path in paths.items():
        
        if verbose: print('\nRIDER: {}'.format(rider))
        
        # Skip riders that are already employed in deliveries.
        if path['active']: continue
        
        # Update the nodes with the new coordinates.
        nodes = path['nodes'].copy()
        nodes.append(node_coord)
        
        # Time matrix.
        time_matrix = compute_time_matrix(nodes)
        
        # Time windows.
        time_windows = path['time_windows'].copy()
        negative_window = False
        for idx, wdw in enumerate(time_windows):
            if idx == 0: continue
            
            negative_window = (wdw[1] - time) < inputs['prep_time']
            if negative_window: break
            
            time_windows[idx] = (inputs['prep_time'], wdw[1] - time)
        
        if negative_window: continue # If window is negative can skip.      
        time_windows.append((inputs['prep_time'], inputs['delivery_max']))
        
        if verbose: print('\nTime windows: {}'.format(time_windows))
        if verbose: print('For nodes: {}'.format(nodes))

        # Optimize path.
        # Construct data dictionary.
        data = {
            'depot': 0,
            'num_vehicles': 1,
            'time_windows': time_windows,
            'time_matrix': time_matrix
        }
        
        # Run optimizer.
        manager, routing, solution = find_path(data)

        # Get data from solution, routing and manager.
        time_dimension = routing.GetDimensionOrDie('Time')
        vehicle_id = 0
        total_nodes = -1 # Takes into account first node
        new_path = []

        # Route.
        index = routing.Start(vehicle_id)
        while not routing.IsEnd(index):
            total_nodes += 1
            new_path.append(manager.IndexToNode(index))
            #if manager.IndexToNode(index) != 0:
            #    path_nodes.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        
        # Do the old nodes match the new one. 
        # Only additions are allowed, not drops.
        if verbose: print('\n/// NEW PATH')
        if verbose: print('Previous path: {}'.format(path['path']))
        if verbose: print('New path: {}'.format(new_path))
        
        
        match = nodes_match(path['nodes'], new_path)
        if verbose: print('Path accepted: {}\n'.format(match))
        if match:
            orderAssigned = True
            if verbose: print("=> UPDATE PATH")
            paths[rider]['nodes'] = nodes
            paths[rider]['time_windows'] = time_windows
            paths[rider]['path'] = new_path
            
            # COMPUTE MAX TIME
            paths[rider]['must_leave'] = time + inputs['delivery_max'] - solution.Min(time_dimension.CumulVar(index-1))
            paths[rider]['will_return'] = paths[rider]['must_leave'] + solution.Min(time_dimension.CumulVar(index))
        
            if verbose: print('Nodes visited: {}'.format(new_path))
            if verbose: print('Total number of nodes: {}'.format(total_nodes))
            if verbose: print('Current time: {}'.format(time))
            if verbose: print('Must leave at: {}'.format(paths[rider]['must_leave']))
            if verbose: print('Will return at: {}'.format(paths[rider]['will_return']))
            if verbose: print('\n//////////////////\n')
            break
        
        
    return paths

In [197]:
def nodes_match(nodes, new_nodes):
    for idx, _ in enumerate(nodes):
        if idx not in new_nodes:
            return False
    
    return True

In [198]:
def describe_path(rider, path):
    print('Rider {}'.format(rider))
    print('Status {}'.format(path['active']))
    
    all_path = 'Hub => '
    for node in path['path'][1:]:
        all_path += 'Node {} => '.format(node)
    all_path += 'Hub'
    
    print('Time on path: {}'.format(path['will_return'] - path['must_leave']))

In [199]:
hub_coord = (0,0)
open_nodes = []
paths = init_paths(num_riders, hub_coord)
verbose = False
for time in range(timeframe+1):
    if verbose: print('Current time: {}'.format(time))
    
    # Decide on rider's next action: START, WAIT.
    for rider, path in paths.items():    
        # Has returned
        if path['will_return'] == time:
            paths[rider]['must_leave'] = float('inf')
            paths[rider]['will_return'] = float('inf')
            paths[rider]['active'] = False
            paths[rider]['completed'] += 1
            
        if path['active']: 
            if verbose: print(rider, "EMPLOYED")
            continue
            
        if path['must_leave'] > time:
            if verbose: print(rider, 'WAIT')
        else:
            paths[rider]['active'] = True
            if verbose: print(rider, 'START')
    
    # Check if a rider is back from its path.
            
    # Check for new orders.
    for order in order_times:
        # Assign new order to a rider's path.
        if order == time:
            if verbose: print('New order')
            node_coord = random_node()
            paths = assign_path(paths, node_coord, time, inputs)
            
            # Compute max 
            
            # If cannot assign, send to open orders.

KeyError: 'nodes'

In [215]:
# Inputs.
num_orders = 5
timeframe = 10 # minutes
num_riders = 2

cooking_time = 5
delivery_max = 30

inputs = {
    'time': 0,
    'hub_coord': (0,0),
    'prep_time': 5,
    'delivery_max': 30,
    'verbose': True
}

# ---------------------------
orders = [random.randrange(timeframe) for x in range(num_orders)] # Can be improved with density function
orders.sort()
# ---------------------------

open_nodes = []
paths = init_paths(num_riders, hub_coord)
while 100 >= inputs['time']:
    print('time', inputs['time'])
    # Update riders active status.
    # - False, waiting to leave
    # - True, active on a path
    for rider, path in paths.items():
        # Rider becomes active.
        if path['must_leave'] == inputs['time']:
            paths[rider]['active'] = True
            
        # Rider becomes inactive, reset all paths specifics.
        if path['will_return'] == inputs['time']:
            paths[rider]['nodes_coord'] = [(0,0)]
            paths[rider]['time_tracker'] = []
            paths[rider]['path'] = []
            paths[rider]['must_leave'] = float('inf')
            paths[rider]['will_return'] = float('inf')
            paths[rider]['active'] = False
            paths[rider]['completed'].append(path['path'])
        
        # Describe the riders path.
        if inputs['verbose'] & path['active']: describe_path(rider, path)
        
    
    for order in orders:
        if order == inputs['time']:
            print('New order')
            order_coord = random_node()
            print(order_coord)
            # Assign order to rider.
            assign_order(order_coord, inputs, paths)
    
    # Update time passaed.
    inputs['time'] += 1
    

time 0
time 1
New order
(4.84500988332132, -1.0820142625509575)
New order coord (4.84500988332132, -1.0820142625509575)
Nodes coordinates [(0, 0), (4.84500988332132, -1.0820142625509575)]
PATH [0, 1, 0]
New order
(-4.0782726567947645, 7.176515025052621)
New order coord (-4.0782726567947645, 7.176515025052621)
Nodes coordinates [(0, 0), (4.84500988332132, -1.0820142625509575), (-4.0782726567947645, 7.176515025052621)]
PATH [0, 1, 2, 0]
time 2
New order
(-3.255027287915512, 1.1773463232447632)
New order coord (-3.255027287915512, 1.1773463232447632)
Nodes coordinates [(0, 0), (4.84500988332132, -1.0820142625509575), (-4.0782726567947645, 7.176515025052621), (-3.255027287915512, 1.1773463232447632)]
PATH [0, 3, 2, 1, 0]
New order
(-3.9432262310661352, -3.7672586155040704)
New order coord (-3.9432262310661352, -3.7672586155040704)
Nodes coordinates [(0, 0), (4.84500988332132, -1.0820142625509575), (-4.0782726567947645, 7.176515025052621), (-3.255027287915512, 1.1773463232447632), (-3.94322

In [216]:
paths

{'r0': {'nodes_coord': [(0, 0)],
  'time_tracker': [],
  'path': [],
  'must_leave': inf,
  'will_return': inf,
  'active': False,
  'completed': [[]]},
 'r1': {'nodes_coord': [(0, 0)],
  'time_tracker': [],
  'path': [],
  'must_leave': inf,
  'will_return': inf,
  'active': False,
  'completed': []}}

In [202]:
def tsp_path(time_matrix):
    manager = pywrapcp.RoutingIndexManager(len(time_matrix), 1, 0) # Time_matrix, num_riders, depot
    routing = pywrapcp.RoutingModel(manager)
    
    # Time callback, gets the time it takes to travel between two nodes.
    def time_callback(from_index, to_index):
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return time_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Search parameters.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    # Solution.
    solution = routing.SolveWithParameters(search_parameters)
    
    return manager, routing, solution

In [203]:
def get_routes(solution, routing, manager):
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

In [214]:
def assign_order(order_coord, inputs, paths):
    print("New order coord", order_coord)
    # Try assign to available riders.
    for rider, path in paths.items():
        if path['active']: continue # Cannot assign order to a rider already employed on a path.
        
        # Create the data to optimize the path.
        time_tracker = path['time_tracker'].copy()
        time_tracker.append(inputs['time'])
        nodes_coord = path['nodes_coord'].copy() 
        nodes_coord.append(order_coord) # Add the node to the established path of the rider.
        time_matrix = compute_time_matrix(nodes_coord) # Update the time matrix with the new nodes.
        
        print('Nodes coordinates', nodes_coord)
        
        manager, routing, solution = tsp_path(time_matrix)
        routes = get_routes(solution, routing, manager)
        
        new_nodes = routes[0][1:-1] # Remove first and last.
        
        # Check if path is within conditions. 
        total_time = 0
        last_node = 0
        must_leave = float('inf')
        for node in new_nodes:   
            total_time += time_matrix[last_node][node]
            last_node = node
            
            # Check if time permits
            time_permits = time_tracker[node - 1] + 30 > inputs['time'] + total_time # Time to get there
            if not time_permits: break
            if time_tracker[node - 1] + 30 - total_time < must_leave:
                must_leave = time_tracker[node - 1] + 30 - total_time
        
        total_time += time_matrix[last_node][0]
        
        # Add the path.
        paths[rider]['nodes_coord'] = nodes_coord
        paths[rider]['time_tracker'] = time_tracker
        paths[rider]['path'] = routes[0]
        print("PATH", routes[0])
        paths[rider]['must_leave'] = must_leave
        paths[rider]['will_return'] = inputs['time'] + total_time
        paths[rider]['active'] = must_leave == inputs['time']          

        # Order has been assigned
        break
        
        # what if he must leave now?
        print('Time for route: {}'.format(total_time))

In [96]:
# When rider comes back reinit

In [108]:
'Run on every new order'
def assiaagn_order(order_coord, inputs):
    # Check if order is assigned
    order_assigned = False
    
    # First tentative to assign order.
    for rider, path in paths.items():
        if path['active']: continue # Cannot assign order to a rider already on path.

        # Create the data to optimize the path.
        nodes = path['nodes'].copy() 
        nodes.append(node_coord) # Add the node to the established path of the rider.
        
        time_matrix = compute_time_matrix(nodes) # Update the time matrix with the new nodes
        
        # Time windows.
        time_windows = path['time_windows'].copy()
        negative_window = False
        for idx, wdw in enumerate(time_windows):
            if idx == 0: continue
            
            negative_window = (wdw[1] - time) < inputs['prep_time']
            if negative_window: break
            
            time_windows[idx] = (inputs['prep_time'], wdw[1] - time)
        
        if negative_window: continue # If window is negative can skip.      
        time_windows.append((inputs['prep_time'], inputs['delivery_max']))

In [ ]:
def vrp_path(data):
    

In [103]:
paths

{'r0': {'nodes': [(0, 0)],
  'time_windows': [(5, 5)],
  'path': [],
  'must_leave': inf,
  'will_return': inf,
  'active': False,
  'completed': 0},
 'r1': {'nodes': [(0, 0)],
  'time_windows': [(5, 5)],
  'path': [],
  'must_leave': inf,
  'will_return': inf,
  'active': False,
  'completed': 0}}

In [78]:
all_completed = 0
for _, path in paths.items():
    all_completed += path['completed']

In [79]:
all_completed

20